## Classical AES used in TLS

In [1]:
#https://stackoverflow.com/a/21928790
import base64
import hashlib
from Crypto import Random
from Crypto.Cipher import AES

class AESCipher(object):

    def __init__(self, key): 
        self.bs = AES.block_size
        self.key = hashlib.sha256(key.encode()).digest()

    def encrypt(self, raw):
        raw = self._pad(raw)
        iv = Random.new().read(AES.block_size) #initialization vector/seed
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return base64.b64encode(iv + cipher.encrypt(raw.encode()))

    def decrypt(self, enc):
        enc = base64.b64decode(enc)
        iv = enc[:AES.block_size]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return self._unpad(cipher.decrypt(enc[AES.block_size:])).decode('utf-8')

    def _pad(self, s):
        return s + (self.bs - len(s) % self.bs) * chr(self.bs - len(s) % self.bs)

    @staticmethod
    def _unpad(s):
        return s[:-ord(s[len(s)-1:])]

## Quantum resistant key-exchange

NewHope specification: https://newhopecrypto.org/data/NewHope_2020_04_10.pdf

Key exchange (page 5): https://eprint.iacr.org/2015/1092.pdf 

In [2]:
#https://github.com/nakov/PyNewHope
from pynewhope import newhope

def convert_key(sharedKey):
    return ''.join([chr(byte) for byte in sharedKey])

In [3]:
#Alice side part: 1/2
alicePrivKey, b_and_seed = newhope.keygen()

In [4]:
#Bob side
bobSharedKey, u_and_r = newhope.sharedB(b_and_seed)

In [5]:
#Alice side part: 2/2
aliceSharedKey = newhope.sharedA(u_and_r, alicePrivKey)

## Comunication

In [6]:
#Alice side
state=AESCipher(convert_key(aliceSharedKey))
message="Hello quantum world!"
secret=state.encrypt(message)
print(secret)

b'uNmjWbzXffOTioS7GrrRIlu1aKGFFYA+TJRY4Vb3UxMvgMQlQc3PAtX4G/Mr/k6E'


In [7]:
#Bob side
state=AESCipher(convert_key(bobSharedKey))
message=state.decrypt(secret)
print(message)

Hello quantum world!
